In [1]:
!pip install recommenders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.4/51.4 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 10.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 355.3/355.3 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.5/31.5 MB 69.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 79.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.2/264.2 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 110.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.0/113.0 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 501.1/501.1 kB 32.6 MB/s e

In [2]:
import pickle
import pandas as pd


from recommenders.evaluation.python_evaluation import (
    catalog_coverage,
    diversity,
    novelty,
    serendipity,

)

## Animelist

In [3]:
DATA_PATH = '/content/drive/MyDrive/Final Project/Codes/animelist-goodbooks-recommendation/animelist/data_sample_split/'

In [ ]:
train_data = pd.read_csv(f'{DATA_PATH}/data_train_full.tsv', sep='\t')
test_data = pd.read_csv(f'{DATA_PATH}/data_test.tsv', sep='\t')

In [ ]:
PREDICTION_PATH = '/content/drive/MyDrive/Final Project/Codes/animelist-goodbooks-recommendation/goodbook-10k/predictions'
all_predictions_svd = pickle.load(open(f'{PREDICTION_PATH}/predictions_svd_opt.pkl', 'rb'))
all_predictions_mpop = pickle.load(open(f'{PREDICTION_PATH}/predictions_pop_opt.pkl', 'rb'))
all_predictions_bpr = pickle.load(open(f'{PREDICTION_PATH}/predictions_bpr_opt.pkl', 'rb'))
all_predictions_gru =  pickle.load(open(f'{PREDICTION_PATH}/predictions_gru_opt.pkl', 'rb'))

In [ ]:
all_predictions_svd = all_predictions_svd.sort_values(by=['user_id', 'prediction'], ascending=[True, False])
all_predictions_mpop = all_predictions_mpop.sort_values(by=['user_id', 'prediction'], ascending=[True, False])
all_predictions_bpr = all_predictions_bpr.sort_values(by=['user_id', 'prediction'], ascending=[True, False])
all_predictions_gru = all_predictions_gru.sort_values(by=['user_id', 'prediction'], ascending=[True, False])

In [ ]:
top10_svd = all_predictions_svd.groupby('user_id').head(10).reset_index(drop=True)
top10_mpop = all_predictions_mpop.groupby('user_id').head(10).reset_index(drop=True)
top10_bpr = all_predictions_bpr.groupby('user_id').head(10).reset_index(drop=True)

In [ ]:
filtered_df_gru = all_predictions_gru.merge(train_data, on=['user_id', 'anime_id'], how='left', indicator=True)
filtered_df_gru = filtered_df_gru[filtered_df_gru['_merge'] == 'left_only']
filtered_df_gru = filtered_df_gru.drop(columns=['_merge'])

In [ ]:
len(all_predictions_gru), len(filtered_df_gru)

(10000000, 9014030)

In [ ]:
len(all_predictions_gru) - len(train_data)

9014030

In [ ]:
top10_gru = filtered_df_gru.groupby('user_id').head(10).reset_index(drop=True)

In [ ]:
animelist_svd_eval = {
    'coverage': catalog_coverage(train_data, top10_svd, col_user="user_id", col_item="anime_id"),
    'diversity': diversity(train_data, top10_svd, col_user="user_id", col_item="anime_id"),
    'novelty': novelty(train_data, top10_svd, col_user="user_id", col_item="anime_id"),
    'serendipity': serendipity(train_data, top10_svd, col_user="user_id", col_item="anime_id")
}

In [ ]:
animelist_mpop_eval = {
    'coverage': catalog_coverage(train_data, top10_mpop, col_user="user_id", col_item="anime_id"),
    'diversity': diversity(train_data, top10_mpop, col_user="user_id", col_item="anime_id"),
    'novelty': novelty(train_data, top10_mpop, col_user="user_id", col_item="anime_id"),
    'serendipity': serendipity(train_data, top10_mpop, col_user="user_id", col_item="anime_id")
}

In [ ]:
animelist_bpr_eval = {
    'coverage': catalog_coverage(train_data, top10_bpr, col_user="user_id", col_item="anime_id"),
    'diversity': diversity(train_data, top10_bpr, col_user="user_id", col_item="anime_id"),
    'novelty': novelty(train_data, top10_bpr, col_user="user_id", col_item="anime_id"),
    'serendipity': serendipity(train_data, top10_bpr, col_user="user_id", col_item="anime_id")
}

In [ ]:
animelist_gru_eval = {
    'coverage': catalog_coverage(train_data, top10_gru, col_user="user_id", col_item="anime_id"),
    'diversity': diversity(train_data, top10_gru, col_user="user_id", col_item="anime_id"),
    'novelty': novelty(train_data, top10_gru, col_user="user_id", col_item="anime_id"),
    'serendipity': serendipity(train_data, top10_gru, col_user="user_id", col_item="anime_id")
}

In [ ]:
pd.DataFrame({"SVD":animelist_svd_eval,
              "BPR":animelist_bpr_eval,
              "Mpop": animelist_mpop_eval,
              'GRU': animelist_gru_eval})

,SVD,BPR,Mpop,GRU
coverage,0.431000,0.987000,0.088000,1.000000
diversity,0.766709,0.657023,0.519887,0.670220
novelty,10.112966,9.157680,8.019348,10.373932
serendipity,0.815348,0.709147,0.707451,0.810791


## Goodbooks

In [4]:
DATA_PATH = '/content/drive/MyDrive/Final Project/Codes/animelist-goodbooks-recommendation/goodbooks-10k/data_sample_split/'

In [5]:
train_data = pd.read_csv(f'{DATA_PATH}/data_train_full.tsv', sep='\t')
test_data = pd.read_csv(f'{DATA_PATH}/data_test.tsv', sep='\t')

In [6]:
PREDICTION_PATH = '/content/drive/MyDrive/Final Project/Codes/animelist-goodbooks-recommendation/goodbooks-10k/predictions'
all_predictions_svd = pickle.load(open(f'{PREDICTION_PATH}/predictions_svd_opt.pkl', 'rb'))
all_predictions_mpop = pickle.load(open(f'{PREDICTION_PATH}/predictions_pop_opt.pkl', 'rb'))
all_predictions_bpr = pickle.load(open(f'{PREDICTION_PATH}/predictions_bpr_opt.pkl', 'rb'))
all_predictions_gru =  pickle.load(open(f'{PREDICTION_PATH}/predictions_gru_opt.pkl', 'rb'))

In [7]:
all_predictions_svd = all_predictions_svd.sort_values(by=['user_id', 'prediction'], ascending=[True, False])
all_predictions_mpop = all_predictions_mpop.sort_values(by=['user_id', 'prediction'], ascending=[True, False])
all_predictions_bpr = all_predictions_bpr.sort_values(by=['user_id', 'prediction'], ascending=[True, False])
all_predictions_gru = all_predictions_gru.sort_values(by=['user_id', 'prediction'], ascending=[True, False])

In [8]:
top10_svd = all_predictions_svd.groupby('user_id').head(10).reset_index(drop=True)
top10_mpop = all_predictions_mpop.groupby('user_id').head(10).reset_index(drop=True)
top10_bpr = all_predictions_bpr.groupby('user_id').head(10).reset_index(drop=True)

In [9]:
filtered_df_gru = all_predictions_gru.merge(train_data, on=['user_id', 'book_id'], how='left', indicator=True)
filtered_df_gru = filtered_df_gru[filtered_df_gru['_merge'] == 'left_only']
filtered_df_gru = filtered_df_gru.drop(columns=['_merge'])

In [10]:
len(all_predictions_gru), len(filtered_df_gru)

(10000000, 9537370)

In [11]:
len(all_predictions_gru) - len(train_data)

9537370

In [12]:
top10_gru = filtered_df_gru.groupby('user_id').head(10).reset_index(drop=True)

In [13]:
goodbooks_svd_eval = {
    'coverage': catalog_coverage(train_data, top10_svd, col_user="user_id", col_item="book_id"),
    'diversity': diversity(train_data, top10_svd, col_user="user_id", col_item="book_id"),
    'novelty': novelty(train_data, top10_svd, col_user="user_id", col_item="book_id"),
    'serendipity': serendipity(train_data, top10_svd, col_user="user_id", col_item="book_id")
}

/usr/local/lib/python3.11/dist-packages/recommenders/evaluation/python_evaluation.py:1233: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  item_pair_sim[col_sim].fillna(0, inplace=True)
/usr/local/lib/python3.11/dist-packages/recommenders/evaluation/python_evaluation.py:1348: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  avg_diversity = df_user

In [14]:
goodbooks_mpop_eval = {
    'coverage': catalog_coverage(train_data, top10_mpop, col_user="user_id", col_item="book_id"),
    'diversity': diversity(train_data, top10_mpop, col_user="user_id", col_item="book_id"),
    'novelty': novelty(train_data, top10_mpop, col_user="user_id", col_item="book_id"),
    'serendipity': serendipity(train_data, top10_mpop, col_user="user_id", col_item="book_id")
}

/usr/local/lib/python3.11/dist-packages/recommenders/evaluation/python_evaluation.py:1233: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  item_pair_sim[col_sim].fillna(0, inplace=True)
/usr/local/lib/python3.11/dist-packages/recommenders/evaluation/python_evaluation.py:1348: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  avg_diversity = df_user

In [15]:
goodbooks_bpr_eval = {
    'coverage': catalog_coverage(train_data, top10_bpr, col_user="user_id", col_item="book_id"),
    'diversity': diversity(train_data, top10_bpr, col_user="user_id", col_item="book_id"),
    'novelty': novelty(train_data, top10_bpr, col_user="user_id", col_item="book_id"),
    'serendipity': serendipity(train_data, top10_bpr, col_user="user_id", col_item="book_id")
}

/usr/local/lib/python3.11/dist-packages/recommenders/evaluation/python_evaluation.py:1233: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  item_pair_sim[col_sim].fillna(0, inplace=True)
/usr/local/lib/python3.11/dist-packages/recommenders/evaluation/python_evaluation.py:1348: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  avg_diversity = df_user

In [16]:
goodbooks_gru_eval = {
    'coverage': catalog_coverage(train_data, top10_gru, col_user="user_id", col_item="book_id"),
    'diversity': diversity(train_data, top10_gru, col_user="user_id", col_item="book_id"),
    'novelty': novelty(train_data, top10_gru, col_user="user_id", col_item="book_id"),
    'serendipity': serendipity(train_data, top10_gru, col_user="user_id", col_item="book_id")
}

/usr/local/lib/python3.11/dist-packages/recommenders/evaluation/python_evaluation.py:953: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reco_df[col_relevance] = 1.0
/usr/local/lib/python3.11/dist-packages/recommenders/evaluation/python_evaluation.py:1233: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the origina

In [18]:
pd.DataFrame({"SVD":goodbooks_svd_eval,
              "BPR":goodbooks_bpr_eval,
              "Mpop": goodbooks_mpop_eval,
              'GRU': goodbooks_gru_eval})

,SVD,BPR,Mpop,GRU
coverage,0.608000,0.996000,0.049000,0.990000
diversity,0.871818,0.845919,0.536410,0.783910
novelty,9.411718,9.401073,7.401752,10.168314
serendipity,0.911065,0.859970,0.837119,0.915841


## MovieLens

In [21]:
DATA_PATH = '/content/drive/MyDrive/Final Project/Codes/animelist-goodbooks-recommendation/ml-1m/data_split'

In [22]:
train_data = pd.read_csv(f'{DATA_PATH}/data_train_full.tsv', sep='\t')
test_data = pd.read_csv(f'{DATA_PATH}/data_test.tsv', sep='\t')

In [23]:
PREDICTION_PATH = '/content/drive/MyDrive/Final Project/Codes/animelist-goodbooks-recommendation/ml-1m/predictions'
all_predictions_svd = pickle.load(open(f'{PREDICTION_PATH}/predictions_svd_opt.pkl', 'rb'))
all_predictions_mpop = pickle.load(open(f'{PREDICTION_PATH}/predictions_pop_opt.pkl', 'rb'))
all_predictions_bpr = pickle.load(open(f'{PREDICTION_PATH}/predictions_bpr_opt.pkl', 'rb'))
all_predictions_gru =  pickle.load(open(f'{PREDICTION_PATH}/predictions_gru_opt.pkl', 'rb'))

In [24]:
all_predictions_svd = all_predictions_svd.sort_values(by=['user_id', 'prediction'], ascending=[True, False])
all_predictions_mpop = all_predictions_mpop.sort_values(by=['user_id', 'prediction'], ascending=[True, False])
all_predictions_bpr = all_predictions_bpr.sort_values(by=['user_id', 'prediction'], ascending=[True, False])
all_predictions_gru = all_predictions_gru.sort_values(by=['user_id', 'prediction'], ascending=[True, False])

In [25]:
top10_svd = all_predictions_svd.groupby('user_id').head(10).reset_index(drop=True)
top10_mpop = all_predictions_mpop.groupby('user_id').head(10).reset_index(drop=True)
top10_bpr = all_predictions_bpr.groupby('user_id').head(10).reset_index(drop=True)

In [26]:
filtered_df_gru = all_predictions_gru.merge(train_data, on=['user_id', 'movie_id'], how='left', indicator=True)
filtered_df_gru = filtered_df_gru[filtered_df_gru['_merge'] == 'left_only']
filtered_df_gru = filtered_df_gru.drop(columns=['_merge'])

In [27]:
len(all_predictions_gru), len(filtered_df_gru)

(20551105, 19885710)

In [28]:
len(all_predictions_gru) - len(train_data)

19885710

In [29]:
top10_gru = filtered_df_gru.groupby('user_id').head(10).reset_index(drop=True)

In [ ]:
movielens_svd_eval = {
    'coverage': catalog_coverage(train_data, top10_svd, col_user="user_id", col_item="movie_id"),
    'diversity': diversity(train_data, top10_svd, col_user="user_id", col_item="movie_id"),
    'novelty': novelty(train_data, top10_svd, col_user="user_id", col_item="movie_id"),
    'serendipity': serendipity(train_data, top10_svd, col_user="user_id", col_item="movie_id")
}

In [ ]:
movielens_mpop_eval = {
    'coverage': catalog_coverage(train_data, top10_mpop, col_user="user_id", col_item="movie_id"),
    'diversity': diversity(train_data, top10_mpop, col_user="user_id", col_item="movie_id"),
    'novelty': novelty(train_data, top10_mpop, col_user="user_id", col_item="movie_id"),
    'serendipity': serendipity(train_data, top10_mpop, col_user="user_id", col_item="movie_id")
}

In [ ]:
movielens_bpr_eval = {
    'coverage': catalog_coverage(train_data, top10_bpr, col_user="user_id", col_item="movie_id"),
    'diversity': diversity(train_data, top10_bpr, col_user="user_id", col_item="movie_id"),
    'novelty': novelty(train_data, top10_bpr, col_user="user_id", col_item="movie_id"),
    'serendipity': serendipity(train_data, top10_bpr, col_user="user_id", col_item="movie_id")
}

In [ ]:
movielens_gru_eval = {
    'coverage': catalog_coverage(train_data, top10_gru, col_user="user_id", col_item="movie_id"),
    'diversity': diversity(train_data, top10_gru, col_user="user_id", col_item="movie_id"),
    'novelty': novelty(train_data, top10_gru, col_user="user_id", col_item="movie_id"),
    'serendipity': serendipity(train_data, top10_gru, col_user="user_id", col_item="movie_id")
}

In [35]:
pd.DataFrame({"SVD":movielens_svd_eval,
              "BPR":movielens_bpr_eval,
              "Mpop": movielens_mpop_eval,
              'GRU': movielens_gru_eval})

,SVD,BPR,Mpop,GRU
coverage,0.148978,0.308317,0.030524,0.566508
diversity,0.721235,0.631233,0.469389,0.714547
novelty,10.039740,9.380321,8.264081,10.806049
serendipity,0.791191,0.717743,0.704841,0.804573
